In [1]:
%%time
from algoutils import *

raw_courses = read('data/courses.txt')
raw_industries = read('data/industries.txt')
raw_job_industries = read('data/job_industries.txt')

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 11 ms


In [2]:
%%time
# note: enchant module and library should be installed
from cleaning import clean

clean_courses = clean(raw_courses, style='course')
clean_industries = clean(raw_industries, style='industry')
clean_job_industries = clean(raw_job_industries, style='job title')

CPU times: user 13.1 s, sys: 96 ms, total: 13.2 s
Wall time: 13.1 s


In [48]:
%%time
# note: nltk module and nltk.wordnet corpus should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

CPU times: user 60 ms, sys: 24 ms, total: 84 ms
Wall time: 68.4 ms


In [49]:
sorted(course_clusters)[:10]

['ACCOUNTANCY',
 'ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURAL',
 'AGRICULTURE',
 'ANIMATION',
 'APPLIED',
 'ARCHITECTURAL']

In [50]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BPO',
 'BROKERAGE',
 'BUILDING']

In [51]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALL',
 'CENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION']

In [61]:
%%time

import analysis
reload(analysis)
from analysis import *

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = LikelihoodMatrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

CPU times: user 36 ms, sys: 8 ms, total: 44 ms
Wall time: 40.4 ms


In [62]:
CourseIndustry.dataframe.head()

,ENGINE,REPAIR,HARDWARE,PRODUCT,HOSPITALITY,LINE,CAFE,CUSTOMER,OFFICE,POWER,...,DEVELOPMENT,CITY,PLANT,PUBLISHING,SALES,FABRICATION,HEALTH,ASSISTANT,DISTRIBUTOR,BROKERAGE
LINGUISTICS,0,0,0,0,0,0.111282,0,0,0,0,...,0.000000,0.000000,0,0.020568,0,0,0,0.045183,0.042346,0
STUDIES,0,0,0,0,0,0.000000,0,0,0,0,...,0.000000,0.000000,0,0.000000,0,0,0,0.000000,0.000000,0
HARDWARE,0,0,1,0,0,0.000000,0,0,0,0,...,0.000000,0.000000,0,0.000000,0,0,0,0.000000,0.000000,0
HOSPITALITY,0,0,0,0,1,0.000000,0,0,0,0,...,0.000000,0.108406,0,0.000000,0,0,0,0.000000,0.000000,0
ELEMENTARY,0,0,0,0,0,0.000000,0,0,0,0,...,0.112116,0.000000,0,0.000000,0,0,0,0.000000,0.000000,0


In [63]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.6 ms


In [66]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.07 ms


In [67]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.38 ms


ACCOUNTING      3.000000
ACCOUNTANCY     1.778375
BUSINESS        1.000000
AGRIBUSINESS    0.751409
NURSING         0.081877
MARINE          0.072866
ENGINEER        0.060532
MEDICINE        0.059967
ADVERTISING     0.056990
ENGINEERING     0.049126
Name: BUSINESS, dtype: float64

In [68]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.96 ms


RESEARCH       2.711620
INFORMATION    1.438259
TECHNOLOGY     1.340372
CONSULTING     1.205249
CONSULTANCY    1.080940
AGENCY         0.115105
Name: SCIENCE, dtype: float64

In [69]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.27 ms


In [70]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 849 µs


In [71]:
# rank recommendations with the highest percentage
matches.nlargest(5)

RESEARCH       77.086372
MEDICAL         8.036202
ELECTRICAL      6.153122
ELECTRONICS     3.276949
CALLCENTER      2.002045
dtype: float64

In [72]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.15 ms


1.1956868252289226

In [73]:
%%time

# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = LikelihoodMatrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 39.6 ms


In [74]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

0.0

In [75]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'ARCHITECTURE')

0.0

In [76]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'TECHNOLOGY')

0.0

In [77]:
# after adding a match to course and a job, recommendation score should go up
CourseJob.add_match('COMPUTER SCIENCE', 'TECHNOLOGY')
CourseJob.add_match('PROGRAMMING', 'COMPUTER')

In [78]:
CourseJob.recommendation_score('PROGRAMMING', 'COMPUTER')

1.5123227260112406